# 🎮 Battle City + DreamerV3 Training

Train a world-model based RL agent (DreamerV3) to play Battle City on NES.

**Requirements:**
- GPU Runtime (T4 recommended)
- ~10GB disk space

## 1. Install System Dependencies

In [ ]:
# SDL2 for pygame/rendering (optional for headless training)
!apt-get update && apt-get install -y libsdl2-dev libsdl2-image-dev libsdl2-mixer-dev libsdl2-ttf-dev freetype2-demos

## 2. Install Python Dependencies

In [ ]:
# Battle City dependencies
!pip install gymnasium stable-baselines3[extra] nes-py opencv-python shimmy

# DreamerV3 dependencies
!pip install jax[cuda12] jaxlib portal elements ruamel.yaml cloudpickle

# Verify JAX GPU
import jax
print(f"JAX devices: {jax.devices()}")

## 3. Clone Project Repository

In [ ]:
import os

# Mount Google Drive for persistent checkpoints
from google.colab import drive
drive.mount('/content/drive')

# Clone your repository (replace with your repo URL)
REPO_URL = "https://github.com/Nikolay1221/battle-city-dreamer.git"  # <-- CHANGE THIS

if not os.path.exists('/content/battle-city'):
    # If you have a private repo, use SSH or token
    !git clone {REPO_URL} /content/battle-city
else:
    print("Repository already cloned")

os.chdir('/content/battle-city')
!ls -la

## 4. Verify Battle City Environment

In [ ]:
import sys
sys.path.insert(0, '/content/battle-city')

from battle_city_env import BattleCityEnv

# Quick test
env = BattleCityEnv(render_mode='rgb_array')
obs, info = env.reset()
print(f"Observation shape: {obs.shape}")
print(f"Action space: {env.action_space}")

# Test step
obs, reward, done, truncated, info = env.step(1)  # Move up
print(f"Reward: {reward}, Done: {done}")
env.close()
print("\n✅ Battle City environment works!")

## 5. Verify DreamerV3 Environment Wrapper

In [ ]:
from embodied.envs.battlecity import BattleCity

env = BattleCity('stage0')
print(f"Obs space: {env.obs_space}")
print(f"Act space: {env.act_space}")

# Test reset
obs = env.step({'action': 0, 'reset': True})
print(f"Image shape: {obs['image'].shape}")
print(f"Is first: {obs['is_first']}")

env.close()
print("\n✅ DreamerV3 wrapper works!")

## 6. Configure Training

In [ ]:
# Training configuration
LOGDIR = "/content/drive/MyDrive/battlecity_dreamer/logs"
TASK = "battlecity_stage0"
STEPS = 5_000_000
ENVS = 8  # Parallel environments

# Create log directory
import os
os.makedirs(LOGDIR, exist_ok=True)
print(f"Logs will be saved to: {LOGDIR}")

## 7. Start Training 🚀

In [ ]:
# Run DreamerV3 training
!python dreamerv3/main.py \
    --task {TASK} \
    --configs battlecity size12m \
    --logdir {LOGDIR} \
    --run.steps {STEPS} \
    --run.envs {ENVS} \
    --jax.platform cuda

## 8. Monitor Training (TensorBoard)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {LOGDIR}

## 9. Evaluate Trained Agent

In [ ]:
# Run evaluation
!python dreamerv3/main.py \
    --task {TASK} \
    --configs battlecity size12m \
    --logdir {LOGDIR} \
    --script eval_only \
    --run.eval_eps 10